# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing the libraries

### Installing Gymnasium

In [9]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


### Importing the libraries

In [10]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [11]:
class Network(nn.Module):    #Brain of AI

  def __init__(self, state_size, action_size, seed = 42):
      super(Network, self).__init__()
      self.seed = torch.manual_seed(seed)
      self.fc1 = nn.Linear(state_size,64) #First  connection between input layer and fully connected layer (1st layer)...( state size (8 from the website) , number of seeds)
      self.fc2 = nn.Linear(64,64)  # fully connected layer (2nd one)
      self.fc3 = nn.Linear(64,action_size)  #output Layer

  def forward(self, state):  #Method to propgate action from state to output Layer
      x = self.fc1(state)
      x = F.relu(x)  # activation function
      x = self.fc2(x)
      x = F.relu(x)
      return self.fc3(x)

## Part 2 - Training the AI

### Setting up the environment

In [12]:
import gymnasium as gym
env = gym.make('LunarLander-v3') # The Lunar Lander environment was upgraded to v3
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [13]:
learning_rate =5e-4
minibatch_size = 100
discount_factor = 0.99  # Gamma when it is close to 1 considers more the future rewards than the already known
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

### Implementing Experience Replay

In [15]:
class ReplayMemory(object):

    def __init__(self, capacity):
      self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # use GPU then CPU
      self.capacity = capacity
      self.memory = []

    def push(self, event):
      self.memory.append(event)
      if len(self.memory) > self.capacity:  #so it doesn't exceed the memory buffer
        del self.memory[0]   # del the first element in memory


    def sample(self, batch_size):  # randomly select a batch from memory
      experiences = random.sample(self.memory, k = batch_size)
      states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None ]) ).float().to(self.device)   # get first element in experiences which is state
      actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None ]) ).long().to(self.device)
      rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None ]) ).float().to(self.device)
      next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None ]) ).float().to(self.device)
      dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None ]).astype(np.uint8) ).float().to(self.device)
      return states, next_states, actions, rewards, dones

### Implementing the DQN clazzaaaas

In [23]:
class Agent():  # how agent interacts with the evnironment

  def __init__(self, state_size, action_size):
   self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # use GPU then CPU
   self.state_size = state_size
   self.action_size = action_size
   self.local_qnetwork = Network(state_size, action_size).to(self.device)  # Q Network Local  ( select the actions)
   self.target_qnetwork = Network(state_size, action_size).to(self.device)  # Q Network Target ( calculate target q values used in training the local Q network )
   self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
   self.memory = ReplayMemory(replay_buffer_size)
   self.t_step = 0    #time step


  def step(self, state, action, reward, next_state, done):
    self.memory.push((state, action, reward, next_state, done))   # to enter the experience to memory
    self.t_step = (self.t_step +1) % 4  # when to learn from experience every 4 steps
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor) # we learn here


  def act(self, state, epsilon = 0.):    # Help choose the best action by using the best understanding of the policy ( by using epslion greedy & explore random actions sometimes)
     # Convert NumPy array to PyTorch tensor
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)

    self.local_qnetwork.eval()
    with torch.no_grad():  # doing prediction (infrence mode)
      action_values = self.local_qnetwork(state)
    self.local_qnetwork.train()  # back to Training mode
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def learn (self, experiences, discount_factor):  # Uses Experiences from memory to update local Q values to Target values
    states, next_states, actions, rewards, dones = experiences
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)  #maximum value in dimension 1 in target network
    q_targets = rewards + (discount_factor * next_q_targets * (1-dones))
    q_expeced = self.local_qnetwork(states).gather(1, actions)
    loss = F.mse_loss(q_expeced, q_targets)
    self.optimizer.zero_grad()
    loss.backward() #back propagation
    self.optimizer.step() #optimization to update paramters
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)


  def soft_update(self, local_model, target_model, interpolation_parameter):   # Update the parameter of Target Q network to blend it with local Q Network
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)  # Updating Target param using method copy_()




### Initializing the DQN agent

In [24]:
agent = Agent(state_size, number_actions) # Intialize the Agent

### Training the DQN agent

In [26]:
number_episodes = 2000
max_number_timesteps_per_episodes = 1000   # so it won't be stuck in the air
epislon_starting_value = 1.0
epislon_ending_value = 0.01
epislon_decay_value = 0.995
epsilon = epislon_starting_value
scores_on_100_episodes = deque( maxlen =100)


for episode in range(1, number_episodes+1 ):
    state, _ = env.reset()  #rest environmet to initial state
    score = 0    # the reward
    for t in range(0, max_number_timesteps_per_episodes):
      action = agent.act(state, epsilon)  #Select an action
      next_state, reward, done, _, _ = env.step(action)   #The _ is for telling to discard this we don't want it
      agent.step(state, action, reward, next_state, done)  #THIS WILL TRAIN THE AGENT TO LAND BETTER
      state = next_state
      score += reward
      if done:
        break  #Finish the episode
    scores_on_100_episodes.append(score) #score of the last episode that just finished
    epsilon = max(epislon_ending_value, epislon_decay_value * epsilon)
    print('\r Episode {}\tAverage Score: {:.2f}'.format(episode,np.mean(scores_on_100_episodes)), end = "")
    if episode % 100 ==0:
      print('\r Episode {}\tAverage Score: {:.2f}'.format(episode,np.mean(scores_on_100_episodes)))
    if np.mean(scores_on_100_episodes)>= 200.0:
      print('\n Environment solved in {:d} episodes! \tAverage Score: {:.2f}'.format(episode-100 ,np.mean(scores_on_100_episodes)))
      torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
      break





 Episode 100	Average Score: -175.33
 Episode 200	Average Score: -91.23
 Episode 300	Average Score: -40.30
 Episode 400	Average Score: -19.51
 Episode 500	Average Score: 15.07
 Episode 600	Average Score: 120.27
 Episode 679	Average Score: 200.14
 Environment solved in 579 episodes! 	Average Score: 200.14


## Part 3 - Visualizing the results

In [27]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display

def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()